In [1]:
# Load the data
# Clean the data
# Feature Enginnering
# Preproccessing
# Modelling
# RandomSearching
# GridSearchings
# Final Project Repat

In [2]:
import random
import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn
from sklearn.preprocessing import (
    StandardScaler,
    RobustScaler,
    MinMaxScaler,
    MaxAbsScaler,
    OneHotEncoder,
    LabelEncoder,
    Normalizer,
)
from sklearn.model_selection import cross_val_score,train_test_split
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LogisticRegression,LogisticRegressionCV
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import GradientBoostingRegressor,AdaBoostRegressor,VotingRegressor,BaggingRegressor,RandomForestRegressor
from sklearn.svm import SVR
from sklearn.ensemble import BaggingRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.metrics import mean_absolute_error,mean_squared_error,accuracy_score,precision_score,f1_score,recall_score
import pickle
np.random.seed(42)

In [3]:
import wandb
PROJECT_NAME = 'House-Prices-Advanced-Regression-Techniques-V2'
def valid(model,X,y,val=False):
    y_test = y
    y_preds = model.predict(X)
    if val is True:
        results = {
            'Val Accuracy':model.score(X,y),
            'Val MAE':mean_absolute_error(y_test,y_preds),
            'Val MSE':mean_squared_error(y_test,y_preds),
        }
    else:
        results = {
            'Accuracy':model.score(X,y),
            'MAE':mean_absolute_error(y_test,y_preds),
            'MSE':mean_squared_error(y_test,y_preds),
        }
    return results
def fit(model,X_train,X_test,y_train,y_test,name):
    wandb.init(project=PROJECT_NAME,name=name)
    model = model
    model.fit(X_train,y_train)
    print('Logging')
    wandb.log(valid(model,X_train,y_train))
    wandb.log(valid(model,X_test,y_test,True))
    print('Saving')
    pickle.dump(model,open(f'./models/model-{name}.pkl','wb'))
    print('Making Submissions')
    make_submission(model,name)
    return model
def make_submission(model,name):
    data = pd.read_csv('./test.csv')
    print(data.columns)
    for col_name,col_missing_num in zip(data.columns,data.isna().sum()):
        try:
            data[col_name].fillna(data[col_name].median(),inplace=True)
        except:
            data[col_name].fillna(random.choice(data[col_name].dropna().tolist()),inplace=True)
    ids = data['Id']
    cols_to_remove_make_sub = cols_to_remove.copy()
    cols_to_remove_make_sub = list(cols_to_remove_make_sub)
    cols_to_remove_make_sub.remove('SalePrice')
    data = data[cols_to_remove_make_sub]
    data = mct.transform(data)
    preds = model.predict(data)
    df = pd.DataFrame({'Id':ids,'SalePrice':preds})
    df.to_csv(f'./submissions/submisssion-{name}.csv',index=False)
    return df

In [4]:
data = pd.read_csv('./train.csv')
data = data.sample(frac=1)

In [5]:
data.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
892,893,20,RL,70.0,8414,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,MnPrv,NaN,0,2,2006,WD,Normal,154500
1105,1106,60,RL,98.0,12256,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,4,2010,WD,Normal,325000
413,414,30,RM,56.0,8960,Pave,Grvl,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,3,2010,WD,Normal,115000
522,523,50,RM,50.0,5000,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,10,2006,WD,Normal,159000
1036,1037,20,RL,89.0,12898,Pave,NaN,IR1,HLS,AllPub,...,0,NaN,NaN,NaN,0,9,2009,WD,Normal,315500


In [6]:
cols_to_remove = data.dropna(axis=1).columns

In [7]:
data = data.dropna(axis=1)

In [8]:
data.head()

,Id,MSSubClass,MSZoning,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,...,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
892,893,20,RL,8414,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,0,0,0,0,0,2,2006,WD,Normal,154500
1105,1106,60,RL,12256,Pave,IR1,Lvl,AllPub,Corner,Gtl,...,0,0,0,0,0,4,2010,WD,Normal,325000
413,414,30,RM,8960,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,130,0,0,0,0,3,2010,WD,Normal,115000
522,523,50,RM,5000,Pave,Reg,Lvl,AllPub,Corner,Gtl,...,36,0,0,0,0,10,2006,WD,Normal,159000
1036,1037,20,RL,12898,Pave,IR1,HLS,AllPub,Inside,Gtl,...,0,0,0,0,0,9,2009,WD,Normal,315500


In [9]:
object_cols = []
int_cols = []

for col_name,col_dtype in zip(data.columns,data.dtypes):
    if col_dtype == 'int64':
        int_cols.append(col_name)
    if col_dtype == 'object':
        object_cols.append(col_name)
int_cols.remove('SalePrice')

In [10]:
from sklearn.compose import make_column_transformer

In [11]:
X = data.drop('SalePrice',axis=1)
y = data['SalePrice']
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.25)

In [12]:
mct = make_column_transformer(
    (OneHotEncoder(handle_unknown='ignore'),object_cols),
    remainder='passthrough'
)
mct.fit(X_train)
X_train = mct.transform(X_train).toarray()
X_test = mct.transform(X_test).toarray()

In [13]:
# fit(RandomForestRegressor(),X_train,X_test,y_train,y_test,'baseline')

In [18]:
X = data.drop('SalePrice',axis=1)
y = data['SalePrice']
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.25)
mct = make_column_transformer(
    (OneHotEncoder(handle_unknown='ignore'),object_cols),
#     (MaxAbsScaler(),int_cols),
    remainder='passthrough'
)
mct.fit(X_train)
X_train = mct.transform(X_train).toarray()
X_test = mct.transform(X_test).toarray()
fit(GradientBoostingRegressor(),X_train,X_test,y_train,y_test,f'baseline')

Accuracy,0.96799
MAE,10435.32826
MSE,201751483.02849
_runtime,6
_timestamp,1628603234
_step,1
Val Accuracy,0.89858
Val MAE,15954.94385
Val MSE,640896668.14217


Accuracy,▁
MAE,▁
MSE,▁
_runtime,▁▁
_timestamp,▁▁
_step,▁█
Val Accuracy,▁
Val MAE,▁
Val MSE,▁


wandb: wandb version 0.11.2 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


Logging
Saving
Making Submissions
Index(['Id', 'MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street',
       'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig',
       'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType',
       'HouseStyle', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd',
       'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType',
       'MasVnrArea', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual',
       'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1',
       'BsmtFinType2', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating',
       'HeatingQC', 'CentralAir', 'Electrical', '1stFlrSF', '2ndFlrSF',
       'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath',
       'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual',
       'TotRmsAbvGrd', 'Functional', 'Fireplaces', 'FireplaceQu', 'GarageType',
       'GarageYrBlt', 'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual'

GradientBoostingRegressor()